# RAG Sử Dụng Dịch Vụ Azure AI Agent & Semantic Kernel

Đoạn mã này minh họa cách tạo và quản lý một agent Azure AI cho việc tạo nội dung tăng cường truy xuất (RAG) bằng cách sử dụng `Azure AI Agent Service` và `Semantic Kernel`. Agent xử lý các truy vấn của người dùng dựa trên ngữ cảnh được truy xuất và cung cấp các phản hồi chính xác tương ứng.


Sửa lỗi phiên bản SQLite  
Nếu bạn gặp lỗi:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Hãy bỏ chú thích khối mã này ở đầu notebook của bạn:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Nhập Gói
Đoạn mã sau đây nhập các gói cần thiết:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Tạo Nội Dung Tăng Cường Truy Xuất với Semantic Kernel & Azure AI Agent Service

Ví dụ này minh họa cách sử dụng **Azure AI Agent Service** để thực hiện **Tạo Nội Dung Tăng Cường Truy Xuất (RAG)** bằng cách kết hợp mô hình ngôn ngữ với ngữ cảnh cụ thể từ một tài liệu được tải lên.

### Cách Hoạt Động

1. **Tải Tài Liệu**: Một tệp markdown (document.md) chứa thông tin (chính sách bảo hiểm du lịch của Contoso) được tải lên dịch vụ agent.

2. **Tạo Kho Vector**: Tài liệu được lập chỉ mục vào một kho vector để cho phép tìm kiếm ngữ nghĩa trên nội dung của nó.

3. **Cấu Hình Agent**: Một agent được khởi tạo sử dụng mô hình `gpt-4o` với các hướng dẫn nghiêm ngặt sau:
   - Chỉ trả lời các câu hỏi dựa trên nội dung được truy xuất từ tài liệu.
   - Từ chối trả lời nếu câu hỏi nằm ngoài phạm vi.

4. **Tích Hợp Công Cụ Tìm Kiếm Tệp**: `FileSearchTool` được đăng ký với agent, cho phép mô hình tìm kiếm và truy xuất các đoạn thông tin liên quan từ tài liệu đã được lập chỉ mục trong quá trình suy luận.

5. **Tương Tác Người Dùng**: Người dùng có thể đặt câu hỏi. Nếu tìm thấy thông tin liên quan trong tài liệu, agent sẽ tạo câu trả lời có căn cứ.  
   Nếu không, agent sẽ phản hồi rõ ràng rằng tài liệu không chứa đủ thông tin.


### Hàm Chính



Hãy đảm bảo chạy lệnh `az login` bằng Azure CLI trước để cung cấp ngữ cảnh xác thực phù hợp khi sử dụng `DefaultAzureCredential`. Dịch vụ Azure AI Agent không sử dụng khóa API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

# Giới thiệu

Chào mừng bạn đến với tài liệu này! Trong hướng dẫn này, chúng tôi sẽ cung cấp cho bạn thông tin cần thiết để bắt đầu.

## Mục tiêu

- Hiểu rõ các khái niệm cơ bản.
- Làm quen với các công cụ và tài nguyên.
- Xây dựng nền tảng vững chắc để tiến xa hơn.

## Yêu cầu

Trước khi bắt đầu, hãy đảm bảo rằng bạn đã:

1. Cài đặt tất cả các phần mềm cần thiết.
2. Đọc qua tài liệu giới thiệu.
3. Có kiến thức cơ bản về chủ đề này.

[!NOTE] Đây là một ghi chú quan trọng cần lưu ý.

---

## Bắt đầu

### Bước 1: Thiết lập môi trường

Để bắt đầu, hãy làm theo các bước sau:

1. Tải xuống và cài đặt phần mềm từ @@INLINE_CODE_0@@.
2. Cấu hình các biến môi trường như được mô tả trong @@INLINE_CODE_1@@.
3. Kiểm tra cài đặt bằng cách chạy lệnh @@INLINE_CODE_2@@.

[!TIP] Nếu gặp vấn đề, hãy tham khảo phần Câu hỏi thường gặp.

### Bước 2: Chạy ứng dụng mẫu

Sau khi thiết lập xong, bạn có thể chạy ứng dụng mẫu bằng cách:

1. Điều hướng đến thư mục dự án.
2. Chạy lệnh @@INLINE_CODE_3@@.
3. Kiểm tra đầu ra để đảm bảo mọi thứ hoạt động chính xác.

[!IMPORTANT] Đừng quên kiểm tra các quyền truy cập cần thiết.

---

## Kết luận

Chúng tôi hy vọng rằng tài liệu này đã giúp bạn bắt đầu một cách dễ dàng. Nếu bạn có bất kỳ câu hỏi nào, đừng ngần ngại liên hệ với chúng tôi.

Chúc bạn thành công!



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
